In [65]:
import pandas as pd
import numpy as np
import warnings
import os
from typing import Tuple
import json
from data_prep_utils import color_classifier, test_color_mapping, print_color_comparison, CatColorImputer, impute_coat_from_breed, create_metadata_table

warnings.filterwarnings("ignore")

# Show all columns in pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None) # show full content of each cell (no truncation)

DATE_STR = '2025-08-11'

In [66]:
## MAIN PREPROCESSING FUNCTION

def preprocess_cats_data(
        data_type: str,
        directory: str,
        file_name: str,
        output_file_name: str = None ) -> pd.DataFrame:

    """
    Preprocesses cat data for adoptable or adopted cats.

    Parameters:
    - data_type (str): adoptable or adopted
    - directory (str): The directory where the input file is located.
    - file_name (str): The name of the input CSV file.
    - output_file_name (str, optional): The name of the output CSV file to save the processed data.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """

    print(f"Starting preprocessing of '{data_type}' cat data ₍^. .^₎⟆ ...\n")

    # Load data
    df = pd.read_csv(os.path.join(directory, file_name))
    print(f"✓ Data loaded successfully from {os.path.join(directory, file_name)}. Shape: {df.shape}")

    # Convert date columns to datetime
    df['published_at'] = pd.to_datetime(df['published_at'])
    df['status_changed_at'] = pd.to_datetime(df['status_changed_at'])
    df['published_year'] = df['published_at'].dt.year
    print("✓ Date columns converted successfully.")

    # Data type specific steps
    if data_type == 'adoptable':
        
        # Handle midnight times for adoptable cats
        df['accessed_utc'] = pd.to_datetime(df['accessed'], format='mixed', utc=True)
        midnight_mask = df['accessed_utc'].dt.time == pd.Timestamp('00:00:00').time()
        df.loc[midnight_mask, 'accessed_utc'] = (
            df.loc[midnight_mask, 'accessed_utc'].dt.normalize() +
            pd.Timedelta(hours=23, minutes=59, seconds=59)
        )

        # Calculate time difference
        df['adopted_year'] = df['status_changed_at'].dt.year
        df['days_in_shelter'] = (df['accessed_utc'] - df['published_at']).dt.days
        print('✓ `days_in_shelter` column calculated successfully.')

        # Filter negative `days_in_shelter`
        df = df[df['days_in_shelter'] >= 0].reset_index(drop=True)
        print(f"✓ Drop samples with negative `days_in_shelter` value, data shape: {df.shape}")

    else:
        # Filter data before 2020
        df = df[df['published_year'] > 2019]
        print(f"✓ Filtered adopted cats published before 2020, data shape: {df.shape}")

        # Calculate time difference
        df['days_to_adopt'] = (df['status_changed_at'] - df['published_at']).dt.days
        print(f"✓ `days_to_adopt` column calculated successfully.")

        # Filter negative `days_to_adopt`
        df = df[df['days_to_adopt'] >= 0]
        df = df.reset_index(drop=True)
        print(f"✓ Drop samples with negative `days_to_adopt` value, data shape: {df.shape}")


    # Impute coat from breed column
    df['coat'] = df['breeds_primary'].apply(impute_coat_from_breed)
    print("✓ Coat length (short, medium, long, medium) successfully imputed from primary breed column.")

    # Replace null values with 'Unknown' to capture missing info provided on listing by publisher
    df['env_children'] = np.where(df['env_children'].isna(), 'Unknown', df['env_children'])
    df['env_dogs'] = np.where(df['env_dogs'].isna(), 'Unknown', df['env_dogs'])
    df['env_cats'] = np.where(df['env_cats'].isna(), 'Unknown', df['env_cats'])
    df['is_character_provided'] = ~df['tags'].isna()
    print("✓ Replace null values with 'Unknown' to capture missing info provided on listing by publisher.")

    # Standardize and augment color categories
    df['color_str'] = df[['colors_primary', 'colors_secondary', 'colors_tertiary']].fillna('').apply(
        lambda x: ' | '.join(filter(None, x)), axis=1
    )
    df['cleaned_color'] = df['color_str'].apply(color_classifier)
    df['cleaned_color'] = df['cleaned_color'].replace('', np.nan)
    print("✓ Color categories standardized successfully considering primary, secondary and tertiary colors.")

    # Impute color based on description
    imputer = CatColorImputer()
    results_json, _ = imputer.validate_imputation(df=df, test_fraction=0.5)

    # Create imputed color column
    df['imputed_color'] = df.apply(
        lambda row: imputer.extract_color_from_description(row['description'])
        if pd.isna(row['cleaned_color']) else row['cleaned_color'],
        axis=1
    )

    # Number of colors imputed
    imputed_count = int(df['cleaned_color'].isna().sum() - df['imputed_color'].isna().sum())
    print(f"✓ {imputed_count}/{df['cleaned_color'].isna().sum()} cats without colour initially have been imputed.")

    # Derive final color (standardized + imputed)
    df['color'] = np.where(
        df['cleaned_color'].isna(),
        df['imputed_color'],
        df['cleaned_color']
    )
    df['color'] = np.where(df['color'].isna(), 'Unknown', df['color'])
    print("✓ Final color column derived from cleaned and imputed colours successfully.")

    # Derive other useful other features
    color_cat_mapping = {
        'Black': ['Black', 'Black & White / Tuxedo'],
        'Brown / Chocolate': ['Tabby (Brown / Chocolate)', 'Brown / Chocolate', 'Brown & White'],
        'Gray / Blue / Silver': ['Tabby (Gray / Blue / Silver)', 'Gray / Blue / Silver', 'Gray & White', 'Blue Cream', 'Smoke'],
        'Orange / Red': ['Tabby (Orange / Red)', 'Orange / Red', 'Orange & White'],
        'White / Cream': ['White', 'Cream / Ivory', 'White / Cream', 'Buff & White', 'Tabby (Buff / Tan / Fawn)', 'Buff / Tan / Fawn'],
        'Calico / Tortie': ['Calico', 'Dilute Calico', 'Tortoiseshell', 'Dilute Tortoiseshell', 'Torbie'],
        'Point Colors': ['Seal Point', 'Lilac Point', 'Cream Point', 'Flame Point', 'Blue Point', 'Chocolate Point'],
        'Special Patterns': ['Tabby (Tiger Striped)', 'Tabby (Leopard / Spotted)']
    }
    reverse_mapping = {}
    for category, colors in color_cat_mapping.items():
        for color in colors:
            reverse_mapping[color] = category
    df['color_category'] = df['color'].map(reverse_mapping)
    df['color_category'] = np.where(df['color_category'].isna(), 'Unknown', df['color_category'])
    print("✓ Broader colour categories created from final color successfully.")

    # Drop unnecessary columns
    COLS_TO_DROP = ['type', 'species', 'distance', 'color_str']
    df = df.drop(columns=COLS_TO_DROP)
    df = df.reset_index(drop=True)
    print("✓ Dropped unnecessary columns - 'type', 'species', 'distance', 'color_str'.")

    # Save processed data to CSV if output file name is provided
    if output_file_name:
        df.to_csv(os.path.join(directory, output_file_name), index=False)
        print(f"✓ Processed data saved successfully to: {os.path.join(directory, output_file_name)}.")

    print("\nPreprocessing completed  ദ്ദി(˵ •̀ ᴗ - ˵ ) ✧ ")


    # print('\nColour cleaning logs:\n')

    # print('\n------------ Colour Standardisation Mapping ------------')
    # test_color_mapping(df)
    print('\n------------ Colour Imputer Test Results ------------')
    print(json.dumps(results_json, indent=4))
    print("\n------------  Colour Distribution Comparison ------------")
    print_color_comparison(df, original_col_to_compare='colors_primary')
    return df

In [49]:
# read processed datasets into memory if already exists
# directory = os.path.join('cat', DATE_STR, 'adoptable')
# adoptable_cats = pd.read_csv(directory + '/processed_adoptable_cats.csv')

# directory = os.path.join('cat', DATE_STR, 'adopted')
# adopted_cats = pd.read_csv(directory + '/processed_adopted_cats.csv')

## Adopted Cats

In [67]:
directory = os.path.join('cat', DATE_STR, 'adopted')
adopted_cats = preprocess_cats_data(
    data_type = 'adopted',
    directory=directory,
    file_name='all_adopted_cats.csv',
    output_file_name='processed_adopted_cats.csv',
)

Starting preprocessing of 'adopted' cat data ₍^. .^₎⟆ ...

✓ Data loaded successfully from cat/2025-08-11/adopted/all_adopted_cats.csv. Shape: (872490, 45)
✓ Date columns converted successfully.
✓ Filtered adopted cats published before 2020, data shape: (841778, 46)
✓ `days_to_adopt` column calculated successfully.
✓ Drop samples with negative `days_to_adopt` value, data shape: (841776, 47)
✓ Coat length (short, medium, long, medium) successfully imputed from primary breed column.
✓ Replace null values with 'Unknown' to capture missing info provided on listing by publisher.
✓ Color categories standardized successfully considering primary, secondary and tertiary colors.
✓ 22742/211760 cats without colour initially have been imputed.
✓ Final color column derived from cleaned and imputed colours successfully.
✓ Broader colour categories created from final color successfully.
✓ Dropped unnecessary columns - 'type', 'species', 'distance', 'color_str'.
✓ Processed data saved successfully to:

In [68]:
adopted_cols_metadata = create_metadata_table(adopted_cats)
display(adopted_cols_metadata)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841776 entries, 0 to 841775
Data columns (total 49 columns):
Memory usage: 2077214375 bytes

Detailed Metadata:


,Column,Non-Null Count,Null Count,Dtype,Category,Values/Range
0,id,841776,0,int64,Numerical,"[37300481, 77730279]"
1,org_id,841776,0,object,Categorical,3859 unique values
2,url,841776,0,object,Categorical,841776 unique values
3,age,841776,0,object,Categorical,"['Young', 'Adult', 'Baby', 'Senior']"
4,gender,841776,0,object,Categorical,"['Male', 'Female']"
5,size,841776,0,object,Categorical,"['Medium', 'Small', 'Large', 'Extra Large']"
6,coat,841776,0,object,Categorical,"['Short', 'Unknown', 'Medium', 'Long', 'Hairless']"
7,name,841764,12,object,Categorical,227116 unique values
8,description,734673,107103,object,Categorical,628042 unique values
9,status,841776,0,object,Categorical,['adopted']


In [69]:
adopted_cats.groupby('color_category')['days_to_adopt'].describe().sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
color_category,,,,,,,,
Unknown,189018.0,30.532288,83.784830,0.0,0.0,4.0,27.0,1900.0
Black,181328.0,53.122127,103.056453,0.0,7.0,21.0,54.0,1991.0
Gray / Blue / Silver,132880.0,46.894484,95.844257,0.0,6.0,18.0,47.0,1933.0
Brown / Chocolate,87842.0,49.618007,98.572781,0.0,7.0,19.0,50.0,1930.0
Calico / Tortie,83725.0,49.890917,100.476085,0.0,7.0,19.0,49.0,1900.0
Orange / Red,77935.0,45.034785,94.750007,0.0,6.0,17.0,44.0,1798.0
White / Cream,50727.0,49.118260,98.805363,0.0,6.0,19.0,49.0,1905.0
Special Patterns,21895.0,51.395707,102.268493,0.0,7.0,19.0,50.0,1464.0
Point Colors,16426.0,33.744003,75.927987,0.0,5.0,12.0,31.0,1449.0


In [70]:
adopted_cats.groupby('color')['days_to_adopt'].describe().sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
color,,,,,,,,
Unknown,189018.0,30.532288,83.784830,0.0,0.0,4.0,27.0,1900.0
Black & White / Tuxedo,96924.0,55.096581,106.793306,0.0,8.0,22.0,56.0,1917.0
Black,84404.0,50.854794,98.542585,0.0,7.0,20.0,52.0,1991.0
Tabby (Brown / Chocolate),77811.0,49.626300,97.817045,0.0,7.0,19.0,50.0,1930.0
Tabby (Gray / Blue / Silver),53074.0,48.925745,96.486610,0.0,7.0,20.0,50.0,1862.0
Gray & White,40569.0,48.561611,98.853274,0.0,6.0,18.0,48.0,1668.0
Gray / Blue / Silver,36361.0,42.498749,91.548131,0.0,5.0,16.0,41.0,1933.0
Tabby (Orange / Red),34818.0,42.083922,87.647045,0.0,6.0,16.0,42.0,1760.0
Calico,31487.0,48.756312,98.894599,0.0,7.0,19.0,49.0,1701.0


## Adoptable Cats

In [71]:
directory = os.path.join('cat', DATE_STR, 'adoptable')
adoptable_cats = preprocess_cats_data(
    data_type = 'adoptable',
    directory=directory,
    file_name='all_adoptable_cats.csv',
    output_file_name='processed_adoptable_cats.csv'
)

Starting preprocessing of 'adoptable' cat data ₍^. .^₎⟆ ...

✓ Data loaded successfully from cat/2025-08-11/adoptable/all_adoptable_cats.csv. Shape: (81057, 45)
✓ Date columns converted successfully.
✓ `days_in_shelter` column calculated successfully.
✓ Drop samples with negative `days_in_shelter` value, data shape: (81057, 49)
✓ Coat length (short, medium, long, medium) successfully imputed from primary breed column.
✓ Replace null values with 'Unknown' to capture missing info provided on listing by publisher.
✓ Color categories standardized successfully considering primary, secondary and tertiary colors.
✓ 4157/30557 cats without colour initially have been imputed.
✓ Final color column derived from cleaned and imputed colours successfully.
✓ Broader colour categories created from final color successfully.
✓ Dropped unnecessary columns - 'type', 'species', 'distance', 'color_str'.
✓ Processed data saved successfully to: cat/2025-08-11/adoptable/processed_adoptable_cats.csv.

Preproces

In [72]:
# pre-requisite: adopted_cats
before_count = len(adoptable_cats)
adoptable_cats = adoptable_cats[~adoptable_cats['id'].isin(adopted_cats['id'])].reset_index(drop=True)
print(f'{before_count - len(adoptable_cats)} adoptable records found already been adopted and have been removed.')

directory = os.path.join('cat', DATE_STR, 'adoptable')
adoptable_cats.to_csv(os.path.join(directory, 'processed_adoptable_cats.csv'), index=False)

2630 adoptable records found already been adopted and have been removed.


In [73]:
adoptable_col_metadata = create_metadata_table(adoptable_cats)
display(adoptable_col_metadata)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78427 entries, 0 to 78426
Data columns (total 51 columns):
Memory usage: 189780650 bytes

Detailed Metadata:


,Column,Non-Null Count,Null Count,Dtype,Category,Values/Range
0,id,78427,0,int64,Numerical,"[669469, 77747364]"
1,org_id,78427,0,object,Categorical,3449 unique values
2,url,78427,0,object,Categorical,78427 unique values
3,age,78427,0,object,Categorical,"['Adult', 'Young', 'Baby', 'Senior']"
4,gender,78427,0,object,Categorical,"['Male', 'Female', 'Unknown']"
5,size,78427,0,object,Categorical,"['Medium', 'Small', 'Large', 'Extra Large']"
6,coat,78427,0,object,Categorical,"['Short', 'Medium', 'Unknown', 'Long', 'Hairless']"
7,name,78427,0,object,Categorical,33487 unique values
8,description,60628,17799,object,Categorical,51979 unique values
9,status,78427,0,object,Categorical,['adoptable']


In [74]:
adoptable_cats.groupby('color_category')['days_in_shelter'].describe().sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
color_category,,,,,,,,
Unknown,25867.0,148.496811,374.559290,0.0,9.0,31.0,109.0,7414.0
Black,17400.0,197.673908,437.666634,0.0,14.0,45.0,180.0,8234.0
Gray / Blue / Silver,11024.0,190.769503,435.377002,0.0,13.0,43.0,173.0,7864.0
Brown / Chocolate,8567.0,184.405276,416.370764,0.0,14.0,43.0,167.5,5904.0
Orange / Red,5652.0,187.575195,492.970881,0.0,11.0,38.0,139.0,8537.0
Calico / Tortie,5306.0,209.493027,455.750109,0.0,14.0,47.0,188.0,6538.0
White / Cream,3512.0,202.302961,436.989923,0.0,13.0,43.0,183.0,8418.0
Special Patterns,703.0,508.631579,870.593255,0.0,26.5,94.0,649.5,4926.0
Point Colors,396.0,132.118687,236.805627,0.0,9.0,32.5,148.0,1574.0


In [77]:
adoptable_cats.groupby('color')['days_in_shelter'].describe().sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
color,,,,,,,,
Unknown,25867.0,148.496811,374.559290,0.0,9.00,31.0,109.00,7414.0
Black,8786.0,159.221261,371.081003,0.0,12.00,37.0,134.00,7725.0
Black & White / Tuxedo,8614.0,236.894358,493.410540,0.0,17.00,54.0,230.00,8234.0
Gray / Blue / Silver,4687.0,134.492213,329.749903,0.0,10.00,34.0,115.00,7864.0
Tabby (Brown / Chocolate),4530.0,239.315673,491.554660,0.0,20.00,59.0,237.00,5668.0
Gray & White,3530.0,214.914731,476.819504,0.0,16.00,54.0,205.50,6170.0
Brown / Chocolate,3256.0,118.011671,283.940954,0.0,10.00,30.0,93.25,4991.0
Tabby (Gray / Blue / Silver),2694.0,254.922049,513.002746,0.0,18.00,58.0,271.00,6139.0
Calico,2165.0,206.157506,502.291634,0.0,12.00,45.0,176.00,6538.0


In [78]:
combined_df = pd.concat([adopted_cats, adoptable_cats], axis=0)
combined_df.to_csv('adopted_and_adoptable_cats.csv', index=False)